# Testing
---
This notebook will be my testing method until one test becomes too big to handle

In [11]:
from qiskit.quantum_info import Pauli,SparsePauliOp,Statevector

In [16]:
psi = Statevector.from_int(4,dims=8)
psi

Statevector([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


In [8]:
pl = Pauli('ZZI')

print(pl.to_matrix(sparse=True))

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 2)	(-1+0j)
  (3, 3)	(-1+0j)
  (4, 4)	(-1+0j)
  (5, 5)	(-1+0j)
  (6, 6)	(1+0j)
  (7, 7)	(1+0j)


In [10]:
H_cut = SparsePauliOp(Pauli('ZZI'))+ SparsePauliOp(Pauli('ZIZ'))

print("H_cut is")
print(H_cut)
print("The sparse representation of H_cut is")
print(H_cut.to_matrix(sparse=True))

H_cut is
SparsePauliOp(['ZZI', 'ZIZ'],
              coeffs=[1.+0.j, 1.+0.j])
The sparse representation of H_cut is
  (0, 0)	(2+0j)
  (1, 1)	0j
  (2, 2)	0j
  (3, 3)	(-2+0j)
  (4, 4)	(-2+0j)
  (5, 5)	0j
  (6, 6)	0j
  (7, 7)	(2+0j)


In [18]:
psi.expectation_value(H_cut).real

-2.0

## Quantum Annealer

In this part of the notebook, we try the Ocean framework, to work with quantum annealers 

In [ ]:
import dimod
# Coefficients of the quadratic term elements (squared or products)
J = {(0,1):1, (0,2):1}
# Coefficients of the linear terms
h = {}
problem = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
print("The problem we are going to solve is:")
print(problem)


The problem we are going to solve is:
BinaryQuadraticModel({0: 0.0, 1: 0.0, 2: 0.0}, {(1, 0): 1.0, (2, 0): 1.0}, 0.0, 'SPIN')


In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
# I need to have an access token 
sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(problem, num_reads=10)
print("The solutions that we have obtained are")
print(result)

ValueError: API token not defined